Defines a shared market-research state to store outputs from each agent.

Creates five specialized AI agents: competitor analysis, customer insights, market trends, pricing strategy, and report writing.

Uses LangGraph to build a DAG workflow connecting these agents sequentially.

Each agent receives the state, adds its analysis, updates logs, and appends key insights.

A central class orchestrates LLM setup, workflow execution, and query handling.

Generates a final executive-ready market research report by synthesizing all agent outputs.

Provides demo, interactive mode, and saving functionality.

NOTE: This is sequential routing, not conditional routing.
The flow is strictly linear (Competitor → Customer → Trends → Pricing → Report) with no branching or decision-based routing. Each agent always runs next in the predefined DAG.

OUTPUT: This system performs end-to-end market research by running multiple specialized AI agents in a fixed LangGraph workflow, generating competitor, customer, trend, and pricing insights, and producing a final executive report.

Here’s how the agents are called:

The graph defines a sequence of nodes: competitor → customer → trends → pricing → report.

When you call workflow.invoke(initial_state), LangGraph automatically triggers each agent in order.

Each agent is simply a Python callable (__call__) that receives the shared state and returns updated state.

LangGraph moves to the next node using the edges you defined (add_edge).

No manual calling needed — the graph engine executes all agents one-by-one until END.

✔ The pretrained LLM produces the insights
✔ LangGraph agents control when and how many times the LLM is called
✔ The routing logic decides which insights to generate

In [ ]:
# Cell 2: Set API key directly
import os

# Replace with your actual API key
os.environ['OPENAI_API_KEY'] = 'A6YPTGzuYNJlrIItM-VnuOPvHJT0lbsCO7FatCTLIzucWWlSp0YS574hDk4Zc36GvcT3BlbkFJN5w0L5iR9PzWiDiA-zQSuHfDGt_eVv3c8yIrxTOWifaU3dPFmkFQXxBh3KLrnQXE6m02VZo90A'
print("✅ API Key set!")

✅ API Key set!


In [5]:
"""
Multi-Agent Market Research System using LangGraph
Advanced research workflow with specialized agents for comprehensive market analysis
"""

import os
from typing import TypedDict, Annotated, List, Literal
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import operator
import json

# ============================================================================
# STATE DEFINITION - Shared research state
# ============================================================================

class MarketResearchState(TypedDict):
    """
    Shared state flowing through all research agents
    """
    research_query: str  # Original research question
    competitor_analysis: str  # Competitor research agent output
    customer_insights: str  # Customer insights agent output
    market_trends: str  # Trend analysis agent output
    pricing_strategy: str  # Pricing analyst output
    final_report: str  # Report writer output
    key_findings: List[str]  # Critical insights collected
    messages: Annotated[List[str], operator.add]  # Agent activity log
    next_agent: str  # Routing control

# This creates a **directed acyclic graph (DAG)** representing the research workflow:
# ```
# START → Competitor → Customer → Trends → Pricing → Report → END
# ============================================================================
# SPECIALIZED RESEARCH AGENTS
# ============================================================================

class CompetitorAnalysisAgent:
    """
    Agent 1: Deep dive into competitive landscape
    """
    def __init__(self, llm):
        self.llm = llm
        self.name = "Competitor Analysis Agent"

    def __call__(self, state: MarketResearchState) -> MarketResearchState:
        """Analyze competitive landscape"""
        system_prompt = """You are an expert Competitive Intelligence Analyst.

            Your mission: Conduct thorough competitor analysis covering:
            - Key competitors and market share
            - Competitor strengths and weaknesses
            - Competitive positioning and differentiation
            - Competitor strategies and recent moves
            - Gaps and opportunities in the market

            Provide actionable intelligence with specific examples and data points.
            Structure your analysis clearly with bullet points for each competitor."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Research Query: {state['research_query']}\n\nProvide comprehensive competitor analysis.")
        ]

        print(f"\n🔍 {self.name} working...")
        response = self.llm.invoke(messages)

        state['competitor_analysis'] = response.content
        state['messages'].append(f"✅ {self.name}: Competitive landscape mapped")

        # Extract key findings
        state['key_findings'].append(f"Competitive Insight: {response.content[:150]}...")

        return state


class CustomerInsightsAgent:
    """
    Agent 2: Analyzes customer needs, behaviors, and segments
    """
    def __init__(self, llm):
        self.llm = llm
        self.name = "Customer Insights Agent"

    def __call__(self, state: MarketResearchState) -> MarketResearchState:
        """Analyze customer landscape"""
        system_prompt = """You are a Customer Research Specialist.

            Your mission: Understand the customer deeply:
            - Target customer segments and personas
            - Customer pain points and needs
            - Buying behaviors and decision criteria
            - Customer journey and touchpoints
            - Unmet needs and desires

            Use behavioral psychology and data-driven insights.
            Create actionable customer profiles."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""
                    Research Query: {state['research_query']}

                    Competitor Context (for reference):
                    {state['competitor_analysis'][:500]}...

                    Provide deep customer insights and segment analysis.
                    """)
                 ]

        print(f"\n👥 {self.name} working...")
        response = self.llm.invoke(messages)

        state['customer_insights'] = response.content
        state['messages'].append(f"✅ {self.name}: Customer profiles created")

        state['key_findings'].append(f"Customer Insight: {response.content[:150]}...")

        return state


class MarketTrendsAgent:
    """
    Agent 3: Identifies market trends, patterns, and future directions
    """
    def __init__(self, llm):
        self.llm = llm
        self.name = "Market Trends Agent"

    def __call__(self, state: MarketResearchState) -> MarketResearchState:
        """Analyze market trends and dynamics"""
        system_prompt = """You are a Market Trends Analyst and Futurist.

            Your mission: Identify and analyze:
            - Current market trends and momentum
            - Emerging technologies and innovations
            - Regulatory and policy changes
            - Macro-economic factors
            - Future market direction (6-18 months)
            - Growth opportunities and threats

            Separate hype from reality. Focus on evidence-based trends.
            Provide specific examples and timeframes."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""
                Research Query: {state['research_query']}

                Market Context:
                - Competitors: {state['competitor_analysis'][:300]}...
                - Customers: {state['customer_insights'][:300]}...

                Analyze market trends and future outlook.
                """)
        ]

        print(f"\n📈 {self.name} working...")
        response = self.llm.invoke(messages)

        state['market_trends'] = response.content
        state['messages'].append(f"✅ {self.name}: Trends identified")

        state['key_findings'].append(f"Trend Insight: {response.content[:150]}...")

        return state


class PricingStrategyAgent:
    """
    Agent 4: Develops pricing analysis and recommendations
    """
    def __init__(self, llm):
        self.llm = llm
        self.name = "Pricing Strategy Agent"

    def __call__(self, state: MarketResearchState) -> MarketResearchState:
        """Analyze pricing landscape and strategy"""
        system_prompt = """You are a Pricing Strategy Expert.

            Your mission: Develop pricing intelligence:
            - Competitor pricing analysis
            - Price sensitivity and elasticity
            - Value-based pricing opportunities
            - Pricing models (subscription, freemium, tiered, etc.)
            - Price positioning recommendations
            - Revenue optimization strategies

            Provide specific pricing ranges and rationale.
            Consider both short-term and long-term pricing strategy."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""
            Research Query: {state['research_query']}

            Full Context:
            - Competitors: {state['competitor_analysis'][:300]}...
            - Customers: {state['customer_insights'][:300]}...
            - Trends: {state['market_trends'][:300]}...

            Develop pricing strategy recommendations.
            """)
        ]

        print(f"\n💰 {self.name} working...")
        response = self.llm.invoke(messages)

        state['pricing_strategy'] = response.content
        state['messages'].append(f"✅ {self.name}: Pricing strategy developed")

        state['key_findings'].append(f"Pricing Insight: {response.content[:150]}...")

        return state


class ReportWriterAgent:
    """
    Agent 5: Synthesizes all research into executive-ready report
    """
    def __init__(self, llm):
        self.llm = llm
        self.name = "Report Writer Agent"

    def __call__(self, state: MarketResearchState) -> MarketResearchState:
        """Create final comprehensive market research report"""
        system_prompt = """You are a Senior Market Research Report Writer.

        Your mission: Synthesize all research into a compelling, executive-ready report.

        Structure your report:

        # EXECUTIVE SUMMARY
        (2-3 paragraphs hitting the key points)

        # MARKET OPPORTUNITY
        - Market size and growth
        - Key opportunities identified

        # COMPETITIVE LANDSCAPE
        - Main competitors and positioning
        - Competitive advantages/gaps

        # CUSTOMER ANALYSIS
        - Target segments
        - Key needs and pain points

        # MARKET TRENDS
        - Major trends affecting the market
        - Future outlook

        # PRICING RECOMMENDATIONS
        - Pricing strategy
        - Revenue model

        # STRATEGIC RECOMMENDATIONS
        (Top 3-5 actionable recommendations)

        # KEY RISKS & MITIGATION

        Make it clear, actionable, and data-driven. Use headers and formatting."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""
            Research Query: {state['research_query']}

            COMPETITOR ANALYSIS:
            {state['competitor_analysis']}

            CUSTOMER INSIGHTS:
            {state['customer_insights']}

            MARKET TRENDS:
            {state['market_trends']}

            PRICING STRATEGY:
            {state['pricing_strategy']}

            Create a comprehensive, executive-ready market research report.
            """)
        ]

        print(f"\n📝 {self.name} writing final report...")
        response = self.llm.invoke(messages)

        state['final_report'] = response.content
        state['messages'].append(f"✅ {self.name}: Report completed")

        return state


# ============================================================================
# LANGGRAPH WORKFLOW ORCHESTRATION
# ============================================================================

class MultiAgentMarketResearch:
    """
    Orchestrates multi-agent market research workflow
    """
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # Initialize LLM
        self.llm = ChatOpenAI(
            model=model,
            temperature=0.7,
            api_key=api_key
        )

        # Initialize all agents
        self.competitor_agent = CompetitorAnalysisAgent(self.llm)
        self.customer_agent = CustomerInsightsAgent(self.llm)
        self.trends_agent = MarketTrendsAgent(self.llm)
        self.pricing_agent = PricingStrategyAgent(self.llm)
        self.report_agent = ReportWriterAgent(self.llm)

        # Build workflow
        self.workflow = self._build_workflow()

    def _build_workflow(self) -> StateGraph:
        """
        Build the research workflow graph

        Flow: Competitor → Customer → Trends → Pricing → Report
        """
        # Create graph
        workflow = StateGraph(MarketResearchState)

        # Add all agent nodes
        workflow.add_node("competitor_analysis", self.competitor_agent)
        workflow.add_node("customer_insights", self.customer_agent)
        workflow.add_node("market_trends", self.trends_agent)
        workflow.add_node("pricing_strategy", self.pricing_agent)
        workflow.add_node("report_writer", self.report_agent)

        # Define sequential flow
        workflow.set_entry_point("competitor_analysis")
        workflow.add_edge("competitor_analysis", "customer_insights")
        workflow.add_edge("customer_insights", "market_trends")
        workflow.add_edge("market_trends", "pricing_strategy")
        workflow.add_edge("pricing_strategy", "report_writer")
        workflow.add_edge("report_writer", END)

        return workflow.compile()

    def research(self, query: str) -> dict:
        """
        Execute full market research workflow

        Args:
            query: Market research question

        Returns:
            Complete research state with final report
        """
        print("\n" + "="*80)
        print("🔬 MULTI-AGENT MARKET RESEARCH SYSTEM ACTIVATED")
        print("="*80)
        print(f"\n📋 Research Query: {query}")
        print("\n⏳ Research in progress...\n")

        # Initialize state
        initial_state = {
            "research_query": query,
            "competitor_analysis": "",
            "customer_insights": "",
            "market_trends": "",
            "pricing_strategy": "",
            "final_report": "",
            "key_findings": [],
            "messages": [],
            "next_agent": ""
        }

        # Execute workflow
        final_state = self.workflow.invoke(initial_state)

        print("\n" + "="*80)
        print("✅ RESEARCH COMPLETE - ALL AGENTS REPORTED")
        print("="*80)

        return final_state


# ============================================================================
# DEMO & UTILITY FUNCTIONS
# ============================================================================

def print_report(result: dict):
    """Pretty print the final report"""
    print("\n" + "="*80)
    print("📊 FINAL MARKET RESEARCH REPORT")
    print("="*80)
    print(result['final_report'])
    print("\n" + "="*80)

    # Show research process
    print("\n📋 Research Process Log:")
    for msg in result['messages']:
        print(f"  {msg}")

    # Show key findings summary
    print("\n🎯 Key Findings Summary:")
    for i, finding in enumerate(result['key_findings'], 1):
        print(f"\n  {i}. {finding}")


def demo_market_research():
    """Demo the market research system"""

    # Get API key
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("\n⚠️  Please set OPENAI_API_KEY environment variable")
        print("   In Colab: os.environ['OPENAI_API_KEY'] = 'your-key'")
        return

    # Initialize system
    research_system = MultiAgentMarketResearch(api_key)

    # Example research query
    query = """
    Conduct comprehensive market research for launching a new AI-powered
    project management tool targeting mid-size tech companies (50-500 employees).
    Include competitor analysis, customer segments, pricing strategy, and market trends.
    """

    # Run research
    result = research_system.research(query)

    # Display results
    print_report(result)

    return result



# Cell 1: Install packages
# !pip install langgraph langchain-openai langchain-core -q



In [6]:

def interactive_research():
    """Interactive mode for custom research queries"""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("\n⚠️  Please set OPENAI_API_KEY")
        return

    research_system = MultiAgentMarketResearch(api_key)

    print("\n" + "="*80)
    print("🔬 MULTI-AGENT MARKET RESEARCH - INTERACTIVE MODE")
    print("="*80)
    print("\nExamples:")
    print("  - Research the electric vehicle charging station market")
    print("  - Analyze the enterprise cybersecurity software market")
    print("  - Study the plant-based food delivery market in urban areas")
    print("\nType 'quit' to exit\n")

    while True:
        try:
            query = input("📋 Your research query: ").strip()

            if not query:
                continue
            if query.lower() == 'quit':
                print("\n👋 Research session ended!\n")
                break

            result = research_system.research(query)
            print_report(result)

            # Ask if user wants to save report
            save = input("\n💾 Save report to file? (y/n): ").strip().lower()
            if save == 'y':
                filename = f"market_research_{hash(query)}.txt"
                with open(filename, 'w') as f:
                    f.write(result['final_report'])
                print(f"✅ Report saved to {filename}")

            print("\n" + "-"*80 + "\n")

        except KeyboardInterrupt:
            print("\n\n👋 Research session ended!\n")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}\n")


# ============================================================================
# QUICK START EXAMPLES
# ============================================================================

def quick_examples():
    """Run quick example queries"""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("\n⚠️  Please set OPENAI_API_KEY")
        return

    research_system = MultiAgentMarketResearch(api_key)

    examples = [
        "Research the market for AI coding assistants for developers",
        "Analyze the online grocery delivery market in Southeast Asia"
    ]

    print("\n🎯 Running Quick Research Examples...\n")

    for i, query in enumerate(examples, 1):
        print(f"\n{'='*80}")
        print(f"EXAMPLE {i}/{len(examples)}")
        print(f"{'='*80}\n")

        result = research_system.research(query)
        print_report(result)

        input("\nPress Enter for next example...")






In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Run main demo
    print("\n🎯 Running Market Research Demo...")
    demo_market_research()

    # Optional: interactive mode
    print("\n" + "="*80)
    choice = input("\nWhat would you like to do next?\n1. Interactive mode\n2. Quick examples\n3. Exit\n\nChoice (1/2/3): ").strip()

    if choice == '1':
        interactive_research()
    elif choice == '2':
        quick_examples()
    else:
        print("\n👋 Goodbye!\n")


🎯 Running Market Research Demo...

🔬 MULTI-AGENT MARKET RESEARCH SYSTEM ACTIVATED

📋 Research Query: 
    Conduct comprehensive market research for launching a new AI-powered
    project management tool targeting mid-size tech companies (50-500 employees).
    Include competitor analysis, customer segments, pricing strategy, and market trends.
    

⏳ Research in progress...


🔍 Competitor Analysis Agent working...

👥 Customer Insights Agent working...

📈 Market Trends Agent working...

💰 Pricing Strategy Agent working...

📝 Report Writer Agent writing final report...

✅ RESEARCH COMPLETE - ALL AGENTS REPORTED

📊 FINAL MARKET RESEARCH REPORT
# EXECUTIVE SUMMARY

This market research report presents a thorough analysis of the potential launch of an AI-powered project management tool targeted at mid-size tech companies, specifically those with 50 to 500 employees. The demand for efficient project management solutions is growing, driven by the increasing complexity of projects and the ne